In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from math import cos, asin, sqrt, pi
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")
bikers=pd.read_csv(r"bikers.csv")
tours=pd.read_csv(r"tours.csv")
bikers_network = pd.read_csv(r"bikers_network.csv")
tour_convoy = pd.read_csv(r"tour_convoy.csv")
locations = pd.read_csv(r"locations.csv")
test_df = pd.read_csv(r"test.csv")
locations = locations.rename({'latitude':'biker_latitude','longitude':'biker_longitude'},axis=1)
tours = tours.rename({'biker_id':'biker_org_id'},axis=1)

In [3]:
train.head(10)

,biker_id,tour_id,invited,timestamp,like,dislike
0,DA44012,QY18771225,0,02-10-2012 15:53:05,0,0
1,DA44012,QU02284248,0,02-10-2012 15:53:05,0,0
2,DA44012,RU29072432,0,02-10-2012 15:53:05,1,0
3,DA44012,SP72478280,0,02-10-2012 15:53:05,0,0
4,DA44012,QS90707377,0,02-10-2012 15:53:05,0,0
5,DA44012,QU32377761,0,02-10-2012 15:53:05,0,0
6,EC36494,RS52676247,0,30-10-2012 01:48:25,0,0
7,EC36494,QU2418051,0,30-10-2012 01:48:28,1,0
8,EC36494,TR03627753,0,30-10-2012 01:49:14,1,0
9,EC36494,QQ0357109,0,30-10-2012 01:48:25,0,0


In [4]:
impute = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
df = pd.DataFrame(bikers.area)
impute = impute.fit(df)
df = impute.transform(df)
bikers['area'] = df

impute = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
df = pd.DataFrame(bikers.gender)
impute = impute.fit(df)
df = impute.transform(df)
bikers['gender'] = df

impute = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
df = pd.DataFrame(bikers.time_zone)
impute = impute.fit(df)
df = impute.transform(df)
bikers['time_zone'] = df

impute = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
df = pd.DataFrame(bikers.member_since)
impute = impute.fit(df)
df = impute.transform(df)
bikers['member_since'] = df
bikers.bornIn = bikers.bornIn.replace('None',1988)

In [5]:
tours['city'].fillna(value=str(tours['city'].mode()), inplace=True)
tours['state'].fillna(value=str(tours['state'].mode()), inplace=True)
tours['country'].fillna(value=str(tours['country'].mode()), inplace=True)
tours['latitude'].fillna(value=int(tours['latitude'].mean()),inplace=True)
tours['longitude'].fillna(value=int(tours['longitude'].mean()),inplace=True)
tours['pincode'].fillna(value=str(tours['pincode'].mode()),inplace=True)

In [6]:
locations['biker_latitude'].fillna(value=locations['biker_latitude'].mean(),inplace=True)
locations['biker_longitude'].fillna(value=locations['biker_longitude'].mean(),inplace=True)

In [7]:
tour_convoy = tour_convoy.replace(np.nan,'X1 X2')
popularity_tour = []
for i in range(tour_convoy.shape[0]):
    if tour_convoy.going.iloc[i] != 'X1 X2' and tour_convoy.maybe.iloc[i] != 'X1 X2' and tour_convoy.not_going.iloc[i]!= 'X1 X2':
        popularity_tour.append(len(tour_convoy.going.iloc[i].split(' '))/(len(tour_convoy.going.iloc[i].split(' '))+len(tour_convoy.maybe.iloc[i].split(' '))+len(tour_convoy.not_going.iloc[i].split(' '))))
    else:
        popularity_tour.append(np.nan)

In [8]:
train=pd.merge(train,bikers,how="left",on='biker_id')
train=pd.merge(train,tours,how='left',on='tour_id')
train['timestamp'] = pd.to_datetime(train.timestamp)
train['tour_date'] = pd.to_datetime(train.tour_date)
train['year_tour'] = train.tour_date.dt.year
train['month_tour'] = train.tour_date.dt.month
train['day_tour'] = train.tour_date.dt.day

In [9]:
train.bornIn = train.bornIn.astype(int)
m = list()
for i in range(train.shape[0]):
    m.append(train.year_tour.iloc[i]-train.bornIn.iloc[i])
    
train['biker_age']=m

In [10]:
train = pd.merge(train,bikers_network,how='left',on='biker_id')
train = pd.merge(train,tour_convoy,how='left',on='tour_id')

In [11]:
train.head(10)

,biker_id,tour_id,invited_x,timestamp,like,dislike,language_id,location_id,bornIn,gender,...,w_other,year_tour,month_tour,day_tour,biker_age,friends,going,maybe,invited_y,not_going
0,DA44012,QY18771225,0,2012-02-10 15:53:05,0,0,id,ID,1990,male,...,2,2012,3,10,22,BC59016464 BI26316229 CG28936374 DE71548202 HA...,GH1846165 DE00950561 DG22121962 CJ14089126 EC7...,DH77634824 HA7808605,CE3449741 CH05495661 CA53207729 CC28554219 CC9...,DG48238803 DI98796217 CI92558795 CF22528816 GF...
1,DA44012,QU02284248,0,2012-02-10 15:53:05,0,0,id,ID,1990,male,...,24,2012,3,10,22,BC59016464 BI26316229 CG28936374 DE71548202 HA...,CI97626992 DC65030005 FJ3212332 BF66080870 BH1...,EC21735584 IC1549713 BF73381076 JD9778463 EB10...,CI23348035 BE61073321 BG03391812 BF11769431 DF...,DJ28943237 BE19223904
2,DA44012,RU29072432,0,2012-02-10 15:53:05,1,0,id,ID,1990,male,...,37,2012,10,26,22,BC59016464 BI26316229 CG28936374 DE71548202 HA...,GI0964033 BB96865463 DJ83005664 DA7654766 CI90...,DC46463441 DB90406675 BE79914576 DA18342187 IC...,BB67170242 IF5781895 BC30475458 DA008058 GB402...,DF51580819 EA04050725 BA08417758 DE50915444 DA...
3,DA44012,SP72478280,0,2012-02-10 15:53:05,0,0,id,ID,1990,male,...,56,2012,6,10,22,BC59016464 BI26316229 CG28936374 DE71548202 HA...,BD59119675 EC65715055 DI6549294 CE97881186 DE3...,BB00969211 EA00810208 GC0797935 EB85121739 CF3...,BE38200374 DB30265952 JG9364175 CA43602169 BJ7...,X1 X2
4,DA44012,QS90707377,0,2012-02-10 15:53:05,0,0,id,ID,1990,male,...,79,2012,6,10,22,BC59016464 BI26316229 CG28936374 DE71548202 HA...,GA3723459 BF91691066 DA9106044 DF20186229 BB65...,BA08417758 IF0700131 DI23300951 DG54313993 DE9...,BB67170242 IF5781895 BC30475458 CG49828467 DF5...,GI123712 CD09856845 DA99357081 BF52779728 EI95...
5,DA44012,QU32377761,0,2012-02-10 15:53:05,0,0,id,ID,1990,male,...,79,2012,6,10,22,BC59016464 BI26316229 CG28936374 DE71548202 HA...,EI1816168 GG5549222 BB94117927 JB1252953 DD673...,DA31153689 BF56407163 DC79992706 DB29498149 GD...,BG69614200 DH11039657 DG61374462 CI84928281 CC...,CE95200649 DD02559951 DD65837945 CI43431637 CB...
6,EC36494,RS52676247,0,2012-10-30 01:48:25,0,0,en,US,1998,female,...,6,2012,4,11,14,CA79929261 EA35599682 BG09007562 DB62489799 BG...,JC430336 CC92571502 DH26818910 BC60599637 DB28...,FJ5790932 DD23155055 BH5822434 EJ3941380 CB545...,CH8428434 DH72198936 IB4080510 BA94982806 IG91...,EE024443 CA72399980 CJ82421152 CD92796738 DE80...
7,EC36494,QU2418051,0,2012-10-30 01:48:28,1,0,en,US,1998,female,...,111,2012,3,11,14,CA79929261 EA35599682 BG09007562 DB62489799 BG...,JI2151653 CF07225225 BC74819181 IF9021736 CB78...,CC77468922 EC49843374 CD90397426 BA30563316 CG...,DB77621817 JI8840288 BH18451861 CH8428434 DF72...,DE35694293 BG63209036 DD74725906 DD85507692 HH...
8,EC36494,TR03627753,0,2012-10-30 01:49:14,1,0,en,US,1998,female,...,22,2012,10,31,14,CA79929261 EA35599682 BG09007562 DB62489799 BG...,JI2151653 CE67676621 DC77257984 HF0190819 FD65...,BD21365746 EB01960082 JF8648795 CB40620608 DC9...,IG2292287 EI9790541 CF74262502 BA90231781 EC80...,CC77468922 JB9828334 BI92311294 CD21908025 BH4...
9,EC36494,QQ0357109,0,2012-10-30 01:48:25,0,0,en,US,1998,female,...,27,2012,10,30,14,CA79929261 EA35599682 BG09007562 DB62489799 BG...,BF40436841 DD06868607 HE0540695 CF56961327 BI0...,DH81650285 BJ45144136 CB73001250 BE91865945 ED...,HF3968424 CD21858766 DA47622499 DC75195402 EC8...,DC54928018 DG82798644 DC64300322 ID7976976 DA2...


In [12]:
num_friends_going = []
num_friends_maybe = []
num_friends_not_going = []
for i in range(train.shape[0]):
    a1 = set(train.going.iloc[i].split(' '))
    b1 = set(train.friends.iloc[i].split(' '))
    c1 = set(train.not_going.iloc[i].split(' '))
    d1 = set(train.maybe.iloc[i].split(' '))
    num_friends_going.append(len(b1&a1))
    num_friends_maybe.append(len(b1&d1))
    num_friends_not_going.append(len(b1&c1))
train['num_friends_going'] = num_friends_going
train['num_friends_maybe'] = num_friends_maybe
train['num_friends_not_going'] = num_friends_not_going

In [13]:
train = train.rename({'invited_x':'invited'},axis=1)

train['timestamp'] = train.timestamp.astype(str)
train.tour_date = train.tour_date.astype(str)

train = pd.merge(train,locations,how='left',on='biker_id')

In [14]:
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

In [15]:
tour_distance = []
for i in range(train.shape[0]):
    lat1 = train.latitude.iloc[i]
    lat2 = train.biker_latitude.iloc[i]
    lon1 = train.longitude.iloc[i]
    lon2 = train.biker_longitude.iloc[i]
    tour_distance.append(distance(lat1,lon1,lat2,lon2))

In [16]:
train['tour_distance']=tour_distance

In [17]:
train = train.drop(columns=['area','biker_org_id','city','state','country','pincode','friends','going','maybe','not_going','invited_y'])

In [18]:
X = train.drop(columns=['biker_id','tour_id','like','dislike'])

In [19]:
categorical = ['language_id','location_id','gender','member_since','timestamp','tour_date','bornIn']

In [20]:
X.head(10)

,invited,timestamp,language_id,location_id,bornIn,gender,member_since,time_zone,tour_date,latitude,...,month_tour,day_tour,biker_age,num_friends_going,num_friends_maybe,num_friends_not_going,Unnamed: 0,biker_latitude,biker_longitude,tour_distance
0,0,2012-02-10 15:53:05,id,ID,1990,male,02-10-2012,480.0,2012-03-10,33.000,...,3,10,22,0,0,3,29638,3.699836,98.429443,15183.844532
1,0,2012-02-10 15:53:05,id,ID,1990,male,02-10-2012,480.0,2012-03-10,-7.767,...,3,10,22,0,0,0,29638,3.699836,98.429443,1838.052226
2,0,2012-02-10 15:53:05,id,ID,1990,male,02-10-2012,480.0,2012-10-26,3.567,...,10,26,22,0,0,0,29638,3.699836,98.429443,28.587100
3,0,2012-02-10 15:53:05,id,ID,1990,male,02-10-2012,480.0,2012-06-10,34.017,...,6,10,22,1,0,0,29638,3.699836,98.429443,4367.398916
4,0,2012-02-10 15:53:05,id,ID,1990,male,02-10-2012,480.0,2012-06-10,3.607,...,6,10,22,0,0,0,29638,3.699836,98.429443,26.869894
5,0,2012-02-10 15:53:05,id,ID,1990,male,02-10-2012,480.0,2012-06-10,3.605,...,6,10,22,0,0,0,29638,3.699836,98.429443,27.262827
6,0,2012-10-30 01:48:25,en,US,1998,female,30-10-2012,240.0,2012-04-11,33.000,...,4,11,14,0,1,1,2939,-20.419233,57.703659,13480.311283
7,0,2012-10-30 01:48:28,en,US,1998,female,30-10-2012,240.0,2012-03-11,-20.162,...,3,11,14,5,6,1,2939,-20.419233,57.703659,35.689446
8,0,2012-10-30 01:49:14,en,US,1998,female,30-10-2012,240.0,2012-10-31,-20.161,...,10,31,14,8,9,1,2939,-20.419233,57.703659,35.778665
9,0,2012-10-30 01:48:25,en,US,1998,female,30-10-2012,240.0,2012-10-30,33.000,...,10,30,14,0,0,0,2939,-20.419233,57.703659,13480.311283


In [21]:
X.describe()

,invited,bornIn,time_zone,latitude,longitude,w1,w2,w3,w4,w5,...,month_tour,day_tour,biker_age,num_friends_going,num_friends_maybe,num_friends_not_going,Unnamed: 0,biker_latitude,biker_longitude,tour_distance
count,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,...,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000,13866.000000
mean,0.042117,1988.554089,128.883600,22.564526,-4.559718,2.242175,1.200418,1.134357,0.695514,1.098659,...,7.771023,17.672508,23.471729,1.124693,0.513703,0.708712,18775.164503,11.088543,41.572751,6223.967400
std,0.200864,10.228496,373.193866,18.357769,84.341366,26.525325,2.805487,2.509509,1.785635,21.714452,...,3.400727,7.974101,10.228156,3.618494,1.661249,5.467252,11037.780040,17.084801,84.849072,6530.670909
min,0.000000,1905.000000,-720.000000,-50.333000,-157.863000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,13.000000,0.000000,0.000000,0.000000,74.000000,-33.873199,-158.188721,0.000118
25%,0.000000,1987.000000,-300.000000,3.607000,-57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,11.000000,18.000000,0.000000,0.000000,0.000000,9282.000000,1.377515,-0.126236,13.339522
50%,0.000000,1992.000000,420.000000,33.000000,-57.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,10.000000,14.000000,21.000000,0.000000,0.000000,0.000000,18399.000000,11.563141,98.678017,2734.719568
75%,0.000000,1994.000000,420.000000,33.000000,98.682000,2.000000,1.000000,1.000000,1.000000,1.000000,...,10.000000,26.000000,25.000000,1.000000,0.000000,0.000000,28242.000000,18.472019,104.865318,13450.206697
max,1.000000,1999.000000,810.000000,60.226000,174.762000,2186.000000,82.000000,85.000000,71.000000,1801.000000,...,12.000000,31.000000,107.000000,168.000000,68.000000,446.000000,38185.000000,57.147476,151.209595,19405.279390


In [22]:
Y = train.like

In [23]:
X_train,X_val,Y_train,Y_val = train_test_split(X,Y,test_size=0.33,random_state=42)

In [24]:
model = CatBoostClassifier(task_type='CPU',iterations=1500,eval_metric='AUC',use_best_model=True)
model_2 = CatBoostClassifier(task_type='CPU',iterations=1500,eval_metric='F1',use_best_model=True)
model.fit(X_train,Y_train,cat_features=categorical,eval_set=(X_val,Y_val))
model_2.fit(X_train,Y_train,cat_features=categorical,eval_set=(X_val,Y_val))

Learning rate set to 0.046069
0:	test: 0.5861863	best: 0.5861863 (0)	total: 393ms	remaining: 9m 49s
1:	test: 0.6337843	best: 0.6337843 (1)	total: 443ms	remaining: 5m 32s
2:	test: 0.6405023	best: 0.6405023 (2)	total: 491ms	remaining: 4m 4s
3:	test: 0.6424738	best: 0.6424738 (3)	total: 548ms	remaining: 3m 24s
4:	test: 0.6452010	best: 0.6452010 (4)	total: 627ms	remaining: 3m 7s
5:	test: 0.7280076	best: 0.7280076 (5)	total: 716ms	remaining: 2m 58s
6:	test: 0.7430523	best: 0.7430523 (6)	total: 778ms	remaining: 2m 45s
7:	test: 0.7436068	best: 0.7436068 (7)	total: 837ms	remaining: 2m 36s
8:	test: 0.7396396	best: 0.7436068 (7)	total: 919ms	remaining: 2m 32s
9:	test: 0.7389558	best: 0.7436068 (7)	total: 1.01s	remaining: 2m 30s
10:	test: 0.7408844	best: 0.7436068 (7)	total: 1.06s	remaining: 2m 24s
11:	test: 0.7411109	best: 0.7436068 (7)	total: 1.11s	remaining: 2m 17s
12:	test: 0.7408912	best: 0.7436068 (7)	total: 1.16s	remaining: 2m 12s
13:	test: 0.7465491	best: 0.7465491 (13)	total: 1.22s	remai

116:	test: 0.8994743	best: 0.8994743 (116)	total: 8.74s	remaining: 1m 43s
117:	test: 0.8995563	best: 0.8995563 (117)	total: 8.8s	remaining: 1m 43s
118:	test: 0.8996619	best: 0.8996619 (118)	total: 8.86s	remaining: 1m 42s
119:	test: 0.8997278	best: 0.8997278 (119)	total: 8.9s	remaining: 1m 42s
120:	test: 0.8997634	best: 0.8997634 (120)	total: 8.95s	remaining: 1m 41s
121:	test: 0.8996533	best: 0.8997634 (120)	total: 8.99s	remaining: 1m 41s
122:	test: 0.8996588	best: 0.8997634 (120)	total: 9.08s	remaining: 1m 41s
123:	test: 0.8997852	best: 0.8997852 (123)	total: 9.15s	remaining: 1m 41s
124:	test: 0.8999268	best: 0.8999268 (124)	total: 9.22s	remaining: 1m 41s
125:	test: 0.8999159	best: 0.8999268 (124)	total: 9.27s	remaining: 1m 41s
126:	test: 0.8998894	best: 0.8999268 (124)	total: 9.32s	remaining: 1m 40s
127:	test: 0.8999691	best: 0.8999691 (127)	total: 9.37s	remaining: 1m 40s
128:	test: 0.8999950	best: 0.8999950 (128)	total: 9.42s	remaining: 1m 40s
129:	test: 0.9000495	best: 0.9000495 (12

228:	test: 0.9050961	best: 0.9051135 (227)	total: 16.2s	remaining: 1m 30s
229:	test: 0.9050878	best: 0.9051135 (227)	total: 16.3s	remaining: 1m 30s
230:	test: 0.9050935	best: 0.9051135 (227)	total: 16.4s	remaining: 1m 29s
231:	test: 0.9051142	best: 0.9051142 (231)	total: 16.4s	remaining: 1m 29s
232:	test: 0.9051116	best: 0.9051142 (231)	total: 16.5s	remaining: 1m 29s
233:	test: 0.9051306	best: 0.9051306 (233)	total: 16.6s	remaining: 1m 29s
234:	test: 0.9051290	best: 0.9051306 (233)	total: 16.6s	remaining: 1m 29s
235:	test: 0.9051288	best: 0.9051306 (233)	total: 16.7s	remaining: 1m 29s
236:	test: 0.9051308	best: 0.9051308 (236)	total: 16.8s	remaining: 1m 29s
237:	test: 0.9050701	best: 0.9051308 (236)	total: 16.8s	remaining: 1m 29s
238:	test: 0.9051449	best: 0.9051449 (238)	total: 16.9s	remaining: 1m 29s
239:	test: 0.9051991	best: 0.9051991 (239)	total: 17s	remaining: 1m 29s
240:	test: 0.9054782	best: 0.9054782 (240)	total: 17.2s	remaining: 1m 29s
241:	test: 0.9056098	best: 0.9056098 (24

340:	test: 0.9072775	best: 0.9073123 (336)	total: 25s	remaining: 1m 24s
341:	test: 0.9072153	best: 0.9073123 (336)	total: 25.1s	remaining: 1m 24s
342:	test: 0.9072568	best: 0.9073123 (336)	total: 25.2s	remaining: 1m 24s
343:	test: 0.9073494	best: 0.9073494 (343)	total: 25.3s	remaining: 1m 25s
344:	test: 0.9073868	best: 0.9073868 (344)	total: 25.4s	remaining: 1m 25s
345:	test: 0.9073447	best: 0.9073868 (344)	total: 25.5s	remaining: 1m 25s
346:	test: 0.9073297	best: 0.9073868 (344)	total: 25.6s	remaining: 1m 25s
347:	test: 0.9073639	best: 0.9073868 (344)	total: 25.7s	remaining: 1m 25s
348:	test: 0.9074067	best: 0.9074067 (348)	total: 25.7s	remaining: 1m 24s
349:	test: 0.9073919	best: 0.9074067 (348)	total: 25.8s	remaining: 1m 24s
350:	test: 0.9074179	best: 0.9074179 (350)	total: 25.9s	remaining: 1m 24s
351:	test: 0.9074135	best: 0.9074179 (350)	total: 25.9s	remaining: 1m 24s
352:	test: 0.9074156	best: 0.9074179 (350)	total: 26s	remaining: 1m 24s
353:	test: 0.9074163	best: 0.9074179 (350)

454:	test: 0.9082912	best: 0.9083503 (423)	total: 34.5s	remaining: 1m 19s
455:	test: 0.9083094	best: 0.9083503 (423)	total: 34.5s	remaining: 1m 19s
456:	test: 0.9082777	best: 0.9083503 (423)	total: 34.6s	remaining: 1m 18s
457:	test: 0.9083026	best: 0.9083503 (423)	total: 34.7s	remaining: 1m 18s
458:	test: 0.9082946	best: 0.9083503 (423)	total: 34.7s	remaining: 1m 18s
459:	test: 0.9082969	best: 0.9083503 (423)	total: 34.8s	remaining: 1m 18s
460:	test: 0.9083304	best: 0.9083503 (423)	total: 34.9s	remaining: 1m 18s
461:	test: 0.9083550	best: 0.9083550 (461)	total: 34.9s	remaining: 1m 18s
462:	test: 0.9083999	best: 0.9083999 (462)	total: 35s	remaining: 1m 18s
463:	test: 0.9083965	best: 0.9083999 (462)	total: 35.1s	remaining: 1m 18s
464:	test: 0.9084365	best: 0.9084365 (464)	total: 35.2s	remaining: 1m 18s
465:	test: 0.9084074	best: 0.9084365 (464)	total: 35.3s	remaining: 1m 18s
466:	test: 0.9084066	best: 0.9084365 (464)	total: 35.4s	remaining: 1m 18s
467:	test: 0.9084126	best: 0.9084365 (46

...

1091:	learn: 0.7899924	test: 0.6557836	best: 0.6632700 (600)	total: 1m 12s	remaining: 27.3s
1092:	learn: 0.7901921	test: 0.6554779	best: 0.6632700 (600)	total: 1m 13s	remaining: 27.2s
1093:	learn: 0.7906037	test: 0.6570229	best: 0.6632700 (600)	total: 1m 13s	remaining: 27.1s
1094:	learn: 0.7906037	test: 0.6570229	best: 0.6632700 (600)	total: 1m 13s	remaining: 27.1s
1095:	learn: 0.7899924	test: 0.6557836	best: 0.6632700 (600)	total: 1m 13s	remaining: 27s
1096:	learn: 0.7899924	test: 0.6557836	best: 0.6632700 (600)	total: 1m 13s	remaining: 26.9s
1097:	learn: 0.7904979	test: 0.6557836	best: 0.6632700 (600)	total: 1m 13s	remaining: 26.9s
1098:	learn: 0.7908034	test: 0.6557836	best: 0.6632700 (600)	total: 1m 13s	remaining: 26.8s
1099:	learn: 0.7914141	test: 0.6554779	best: 0.6632700 (600)	total: 1m 13s	remaining: 26.8s
1100:	learn: 0.7914141	test: 0.6554779	best: 0.6632700 (600)	total: 1m 13s	remaining: 26.7s
1101:	learn: 0.7914141	test: 0.6554779	best: 0.6632700 (600)	total: 1m 13s	remaini

1181:	learn: 0.8009038	test: 0.6563670	best: 0.6632700 (600)	total: 1m 20s	remaining: 21.7s
1182:	learn: 0.8009038	test: 0.6566745	best: 0.6632700 (600)	total: 1m 20s	remaining: 21.6s
1183:	learn: 0.8006027	test: 0.6563670	best: 0.6632700 (600)	total: 1m 20s	remaining: 21.6s
1184:	learn: 0.8009038	test: 0.6569958	best: 0.6632700 (600)	total: 1m 20s	remaining: 21.5s
1185:	learn: 0.8018063	test: 0.6554307	best: 0.6632700 (600)	total: 1m 20s	remaining: 21.4s
1186:	learn: 0.8024072	test: 0.6557377	best: 0.6632700 (600)	total: 1m 20s	remaining: 21.3s
1187:	learn: 0.8024072	test: 0.6557377	best: 0.6632700 (600)	total: 1m 21s	remaining: 21.3s
1188:	learn: 0.8027074	test: 0.6560450	best: 0.6632700 (600)	total: 1m 21s	remaining: 21.2s
1189:	learn: 0.8033074	test: 0.6563526	best: 0.6632700 (600)	total: 1m 21s	remaining: 21.2s
1190:	learn: 0.8033074	test: 0.6557223	best: 0.6632700 (600)	total: 1m 21s	remaining: 21.1s
1191:	learn: 0.8033074	test: 0.6557223	best: 0.6632700 (600)	total: 1m 21s	remai

1275:	learn: 0.8161034	test: 0.6541530	best: 0.6632700 (600)	total: 1m 28s	remaining: 15.5s
1276:	learn: 0.8159006	test: 0.6547842	best: 0.6632700 (600)	total: 1m 28s	remaining: 15.4s
1277:	learn: 0.8159006	test: 0.6541706	best: 0.6632700 (600)	total: 1m 28s	remaining: 15.3s
1278:	learn: 0.8159006	test: 0.6529081	best: 0.6632700 (600)	total: 1m 28s	remaining: 15.3s
1279:	learn: 0.8163062	test: 0.6548009	best: 0.6632700 (600)	total: 1m 28s	remaining: 15.2s
1280:	learn: 0.8163062	test: 0.6544944	best: 0.6632700 (600)	total: 1m 28s	remaining: 15.1s
1281:	learn: 0.8163062	test: 0.6535396	best: 0.6632700 (600)	total: 1m 28s	remaining: 15.1s
1282:	learn: 0.8168034	test: 0.6541706	best: 0.6632700 (600)	total: 1m 28s	remaining: 15s
1283:	learn: 0.8168034	test: 0.6541706	best: 0.6632700 (600)	total: 1m 28s	remaining: 14.9s
1284:	learn: 0.8170974	test: 0.6538642	best: 0.6632700 (600)	total: 1m 28s	remaining: 14.8s
1285:	learn: 0.8166004	test: 0.6535581	best: 0.6632700 (600)	total: 1m 28s	remaini

1366:	learn: 0.8257426	test: 0.6535211	best: 0.6632700 (600)	total: 1m 34s	remaining: 9.21s
1367:	learn: 0.8254518	test: 0.6541530	best: 0.6632700 (600)	total: 1m 34s	remaining: 9.14s
1368:	learn: 0.8254518	test: 0.6541530	best: 0.6632700 (600)	total: 1m 34s	remaining: 9.07s
1369:	learn: 0.8263236	test: 0.6541530	best: 0.6632700 (600)	total: 1m 34s	remaining: 9.01s
1370:	learn: 0.8263236	test: 0.6547842	best: 0.6632700 (600)	total: 1m 35s	remaining: 8.94s
1371:	learn: 0.8260332	test: 0.6554149	best: 0.6632700 (600)	total: 1m 35s	remaining: 8.87s
1372:	learn: 0.8266139	test: 0.6554149	best: 0.6632700 (600)	total: 1m 35s	remaining: 8.81s
1373:	learn: 0.8269041	test: 0.6566604	best: 0.6632700 (600)	total: 1m 35s	remaining: 8.74s
1374:	learn: 0.8269041	test: 0.6563526	best: 0.6632700 (600)	total: 1m 35s	remaining: 8.67s
1375:	learn: 0.8271941	test: 0.6566604	best: 0.6632700 (600)	total: 1m 35s	remaining: 8.61s
1376:	learn: 0.8277737	test: 0.6566604	best: 0.6632700 (600)	total: 1m 35s	remai

1457:	learn: 0.8352623	test: 0.6573034	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.92s
1458:	learn: 0.8352623	test: 0.6573034	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.85s
1459:	learn: 0.8352623	test: 0.6573034	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.78s
1460:	learn: 0.8354680	test: 0.6573034	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.71s
1461:	learn: 0.8354680	test: 0.6576239	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.64s
1462:	learn: 0.8357547	test: 0.6576239	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.57s
1463:	learn: 0.8357547	test: 0.6576239	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.5s
1464:	learn: 0.8357547	test: 0.6579317	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.44s
1465:	learn: 0.8360414	test: 0.6573034	best: 0.6632700 (600)	total: 1m 41s	remaining: 2.37s
1466:	learn: 0.8360414	test: 0.6579317	best: 0.6632700 (600)	total: 1m 42s	remaining: 2.29s
1467:	learn: 0.8357547	test: 0.6579317	best: 0.6632700 (600)	total: 1m 42s	remain

In [25]:
test=pd.merge(test,bikers,how="left",on='biker_id')
test=pd.merge(test,tours,how='left',on='tour_id')
test['timestamp'] = pd.to_datetime(test.timestamp)
test['tour_date'] = pd.to_datetime(test.tour_date)
test['year_tour'] = test.tour_date.dt.year
test['month_tour'] = test.tour_date.dt.month
test['day_tour'] = test.tour_date.dt.day
test.bornIn = test.bornIn.astype(int)
m = list()
for i in range(test.shape[0]):
    m.append(test.year_tour.iloc[i]-test.bornIn.iloc[i])
test['biker_age']=m
test = pd.merge(test,bikers_network,how='left',on='biker_id')
test = pd.merge(test,tour_convoy,how='left',on='tour_id')
num_friends_going = []
num_friends_maybe = []
num_friends_not_going = []
for i in range(test.shape[0]):
    a1 = set(test.going.iloc[i].split(' '))
    b1 = set(test.friends.iloc[i].split(' '))
    c1 = set(test.not_going.iloc[i].split(' '))
    d1 = set(test.maybe.iloc[i].split(' '))
    num_friends_going.append(len(b1&a1))
    num_friends_maybe.append(len(b1&d1))
    num_friends_not_going.append(len(b1&c1))
test['num_friends_going'] = num_friends_going
test['num_friends_maybe'] = num_friends_maybe
test['num_friends_not_going'] = num_friends_not_going
test['timestamp'] = test.timestamp.astype(str)
test.tour_date = test.tour_date.astype(str)
test = pd.merge(test,locations,how='left',on='biker_id')
tour_distance = []
for i in range(test.shape[0]):
    lat1 = test.latitude.iloc[i]
    lat2 = test.biker_latitude.iloc[i]
    lon1 = test.longitude.iloc[i]
    lon2 = test.biker_longitude.iloc[i]
    tour_distance.append(distance(lat1,lon1,lat2,lon2))
test['tour_distance'] = tour_distance

test = test.drop(columns=['area','biker_org_id','city','state','pincode','country','friends','going','maybe','not_going','invited_y'])
test = test.rename({'invited_x':'invited'},axis=1)
test = test.drop(columns=['biker_id','tour_id'])
predict_prob = model.predict_proba(test)
predict_prob_2 = model_2.predict_proba(test)